In [2]:
# comparison notebook
# in snakemake rule, method needs to be passed as a parameter. s.t. it is accessible via: snakemake.params.method

import matplotlib

#%matplotlib inline
# matplotlib.use("Agg")

import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import pandas as pd

sys.path.append(os.getcwd())
import src
from src.noe import compute_NOE_mdtraj, plot_NOE
from src.pca import make_PCA, plot_PCA, getDih
from src.pyreweight import reweight
from src.analyse import getOmega
from src.utils import json_load
from sklearn.decomposition import PCA

from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

import nglview as nv

In [3]:
snakemake.input.NOE_input

In [4]:
tolerance = 0.0
NOEs = []
for f, i in zip(
    snakemake.input.NOE_input, range(len(snakemake.input.NOE_input))
):
    splitted = f.split("/")
    method = splitted[6]
    compound = splitted[3]
    NOE_df = src.noe.read_NOE(f)
    NOE_item = {"method": method, "compound": int(compound)}
    if type(NOE_df) is tuple:
        a, b = NOE_df
        a = a[a["md"] > 0]
        b = b[b["md"] > 0]

        a["within_bounds"] = (a["md"] >= a["lower bound"] - tolerance) & (
            a["md"] <= a["upper bound"] + tolerance
        )
        b["within_bounds"] = (b["md"] >= b["lower bound"] - tolerance) & (
            b["md"] <= b["upper bound"] + tolerance
        )
        NOE_df = a, b
        NOE_item["in_bounds"] = (a.within_bounds).sum() / len(a), (
            b.within_bounds
        ).sum() / len(b)
        NOE_item["in_bounds"] = np.mean(
            [
                (a.within_bounds).sum() / len(a),
                (b.within_bounds).sum() / len(b),
            ]
        )
    else:
        NOE_df = NOE_df[NOE_df["md"] > 0]
        NOE_df["within_bounds"] = (
            NOE_df["md"] >= NOE_df["lower bound"] - tolerance
        ) & (NOE_df["md"] <= NOE_df["upper bound"] + tolerance)
        NOE_item["in_bounds"] = (NOE_df["within_bounds"]).sum() / len(NOE_df)
    NOEs.append(NOE_item)

In [5]:
df = pd.DataFrame(NOEs)

In [6]:
df.groupby("method")["in_bounds"].mean()

In [7]:
df.groupby("method")["in_bounds"].std()

In [8]:
df

In [9]:
df_method = df.pivot(index="compound", columns="method")["in_bounds"]
df_method

In [35]:
# fig, ax = plt.subplots()
ax = df_method.plot.bar()
plt.savefig(snakemake.output.NOE_plot[0])